# Домашнее задание 2

### Реализация градиентного бустинга для классификации

В рамках этой задачи нужно написать градиентный бустинг над решающими деревьями в задаче классификации. В качестве функции потерь предлагается взять **log loss**. Про него можно прочитать подробнее здесь: https://scikit-learn.org/stable/modules/model_evaluation.html#log-loss


$y_i$ это правильный ответ (0 или 1), $\hat{y}_i$ это ваше предсказание

Может показаться, что надо максимизировать функцию $L(\hat{y}, y) = \sum_{i=1}^n y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$,

Это так, но не совсем: лучше максимизировать функцию $L(\hat{y}, y) = \sum_{i=1}^n y_i \log(f(\hat{y}_i)) + (1 - y_i) \log(1 - f(\hat{y}_i))$, где $f(x) = \frac{1}{1 + e^{-x}}$. Благодаря этому у вас не будет ограничений на принимаеммые значения для прогнозов $\hat{y}_i$

### Задание 1

Функцию f(x), предложенную выше, обычно называют **сигмоида** или **сигмоидная функция**. Напишите функцию, вычисляющую значения производной функции f(x).

In [2]:
import numpy as np

In [4]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))


def der_sigmoid(x):
    return np.exp(-x) / (1+np.exp(-x)) ** 2

In [5]:
der_sigmoid(0) == 0.25

True

In [6]:
der_sigmoid(np.array([0, 0])) == np.array([0.25, 0.25])

array([ True,  True])

In [7]:
der_sigmoid(np.log(3)) == 0.1875

True

**Значение для формы:**

In [8]:
print(round(der_sigmoid(np.array([-10, 4.1, -1, 2])).sum() + sigmoid(0.42), 4))

0.9212


Хорошо, теперь мы умеем считать производную функции f, но надо найти производную log loss-а по $\hat{y}$ в первом варианте записи потерь

Напоминание, первый вариант это  $y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$

### Задание 2

Напишите вычисление производной log loss-a

In [15]:
def der_log_loss(y_hat, y_true):
    """
    0 < y_hat < 1
    """
    return y_true / y_hat - (1-y_true)/(1-y_hat)

In [16]:
der_log_loss(0.5, 0) == -2

True

In [11]:
der_log_loss(0.5, 1) == 2

True

In [12]:
der_log_loss(np.array([0.8, 0.8]), np.array([1, 1])) == np.array([1.25, 1.25])

array([ True,  True])

**Значение для формы**

In [17]:
print(round(-sum(der_log_loss((x + 1) / 100., x % 2) for x in range(99)), 2))

69.82


Теперь мы можем воспользоваться формулой производной сложной функции (chain rule) и получить вычисление градиента формулы по второму варианту:

In [18]:
def calc_gradient(y_hat, y_true):
    return der_log_loss(sigmoid(y_hat), y_true) * der_sigmoid(y_hat)

Теперь мы можем написать код градиентного бустинга для классификации

### Задание 3

Допишите класс

In [19]:
from sklearn.base import BaseEstimator # чтобы поддержать интерфейс sklearn
from sklearn.tree import DecisionTreeRegressor # для обучения на каждой итерации

In [45]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters=100, tau=1e-1):
        """
        tree_params_dict - словарь параметров, которые надо использовать при обучении дерева на итерации
        iters - количество итераций
        tau - коэффициент перед предсказаниями деревьев на каждой итерации
        """
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.estimators = []
        curr_pred = 0
        for iter_num in range(self.iters):
            # Нужно найти градиент функции потерь по предсказниям в точке curr_pred
            grad = calc_gradient(curr_pred, y_data)
            # Мы максимизируем, поэтому надо обучить DecisionTreeRegressor с параметрами 
            # tree_params_dict по X_data предсказывать grad
            algo = DecisionTreeRegressor(**self.tree_params_dict).fit(X_data, grad)
            self.estimators.append(algo)
            # все предсказания домножаются на tau и обновляется переменная curr_pred
            curr_pred += self.tau * algo.predict(X_data)
    
    def predict(self, X_data):
        # изначально все предскзания нули
        res = np.zeros(X_data.shape[0])
        for estimator in self.estimators:
            # нужно сложить все предсказания деревьев с весом self.tau
            res += self.tau * estimator.predict(X_data)
            
        return (res > 0).astype(int)

## Проверка качества полученного класса (в самом низу код для формы)

Можете поиграться с параметрами, посмотрим, у кого получится самое лучшее качество

In [46]:
# для оценки качества
from sklearn.model_selection import cross_val_score

# для генерации датасетов
from sklearn.datasets import make_classification

# для сравнения
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [47]:
X_data, y_data = make_classification(n_samples=1000, n_features=10, random_state=42)

In [48]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4
    },
    iters=100,
    tau = 0.1
)

In [42]:
algo.fit(X_data, y_data)

In [49]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='accuracy'))

0.9199999999999999

In [50]:
np.mean(cross_val_score(DecisionTreeClassifier(), X_data, y_data, cv=5, scoring='accuracy'))

0.858

In [51]:
np.mean(cross_val_score(XGBClassifier(), X_data, y_data, cv=5, scoring='accuracy'))

0.9010000000000001

In [52]:
np.mean(cross_val_score(LogisticRegression(), X_data, y_data, cv=5, scoring='accuracy'))

0.8560000000000001

**Значение для формы**

In [53]:
print(round(np.mean(cross_val_score(SimpleGB(
    tree_params_dict={
        'max_depth': 4
    },
    iters=1000,
    tau = 0.01
), X_data, y_data, cv=4, scoring='accuracy')), 3))

0.925
